# 1 Quick Dogs vs. Cats

In [1]:
from fastai.conv_learner import *
PATH = "data/dogscats/"
sz=224
bs=64

In [2]:
# We want to transform our data to fit this particular kind of model
tfms = tfms_from_model(resnet50, sz, aug_tfms=transforms_side_on, max_zoom=1.1)
# We get some data based on the paths, you can rename the train/valid folders by other parameters.
# If you want to submit to kaggle, you need to fill the test_name
data = ImageClassifierData.from_paths(PATH, tfms=tfms, bs=bs)
# By default, it has all of the layers but the last few frozen
# As we use data augment here, we do not set the precompute=True
learn = ConvLearner.pretrained(resnet50, data)
%time learn.fit(1e-2, 3, cycle_len=1)

HBox(children=(IntProgress(value=0, description='Epoch', max=3), HTML(value='')))

epoch      trn_loss   val_loss   accuracy                                                                              
    0      0.046718   0.026802   0.990723  
    1      0.049958   0.029961   0.989746                                                                              
    2      0.031956   0.02434    0.990723                                                                              

Wall time: 14min 27s


[0.024340019, 0.99072265625]

In [3]:
# Now we unfreeze to train the whole model
learn.unfreeze()
# If you are using a bigger deeper model like here resnet50 and resnext101, and on the data side they are very similar to imagenet like these
# ones (size 200~500 pixels), you should add this one line when you unfreeze. 
# It's super important!
learn.bn_freeze(True)
%time learn.fit([1e-5, 1e-4, 1e-2], 1, cycle_len=1)

HBox(children=(IntProgress(value=0, description='Epoch', max=1), HTML(value='')))

epoch      trn_loss   val_loss   accuracy                                                                              
    0      0.028306   0.023633   0.991211  

Wall time: 9min 54s


[0.023633206, 0.9912109375]

In [4]:
%time log_preds, y = learn.TTA()
probs = np.mean(np.exp(log_preds), axis=0)
metrics.log_loss(y, probs), accuracy_np(probs, y)

Wall time: 1min 25s                                                                                                    


(0.018818552416400458, 0.99299999999999999)

In [5]:
# 5 means the augmented data
log_preds.shape

(5, 2000, 2)

In [6]:
probs.shape

(2000, 2)

In [7]:
probs

array([[ 0.99989,  0.00011],
       [ 1.     ,  0.     ],
       [ 0.99964,  0.00036],
       ..., 
       [ 0.00004,  0.99996],
       [ 0.00003,  0.99997],
       [ 0.00001,  0.99999]], dtype=float32)